In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from pathlib import Path
import os
import autogluon
from autogluon.tabular import TabularDataset,TabularPredictor
from sklearn.model_selection import train_test_split
import utils
import geopandas as gpd

In [2]:
current_dir = Path.cwd()
data_dir = current_dir.parent / "data"

In [3]:
gdf_json = utils.merge_geojson_files(data_dir / "train" / "浸水想定区域" / "10_計画規模")
display(gdf_json.head())

305個のgeojsonファイルを全CPUコアを使用して並列読み込みします...
結合完了: 合計 7172706 件のフィーチャー


,A31a_101,A31a_102,A31a_103,A31a_104,A31a_105,A31a_106,A31a_107,A31a_108,geometry
0,8101010001,天塩川,81,北海道開発局,1,None,None,None,"POLYGON ((142.36388 44.26004, 142.36388 44.260..."
1,8101010001,天塩川,81,北海道開発局,1,None,None,None,"POLYGON ((142.38844 44.25612, 142.38844 44.256..."
2,8101010001,天塩川,81,北海道開発局,1,None,None,None,"POLYGON ((142.38719 44.25354, 142.38719 44.253..."
3,8101010001,天塩川,81,北海道開発局,1,None,None,None,"POLYGON ((142.05044 44.83904, 142.05044 44.839..."
4,8101010001,天塩川,81,北海道開発局,1,None,None,None,"POLYGON ((142.06338 44.8405, 142.06338 44.8405..."


In [ ]:
train_df = pd.read_csv(os.path.join('..', 'data', 'train', 'train.csv'))
drop=["reform_place_other","reform_common_area","parking_keiyaku","est_other_distance",'building_name_ruby', 'name_ruby', 'school_ele_code', 'school_jun_code', 'money_hoshou_company', 'free_rent_duration', 'free_rent_gen_timing',]
train_df = train_df.drop(columns=drop)

C:\Users\km20186\AppData\Local\Temp\ipykernel_24972\1774729335.py:1: DtypeWarning: Columns (63) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv(os.path.join('..', 'data', 'train', 'train.csv'))


In [5]:
train_gdf=utils.convert_to_geodataframe(train_df)
train_gdf=utils.spatial_join(train_gdf,gdf_json,predicate='within')

In [29]:
print(train_gdf.shape,train_df.shape)

(363924, 531) (363924, 432)


In [7]:
train_gdf.index.duplicated().sum()

np.int64(0)

In [10]:
# pandasの表示設定
pd.set_option('display.max_columns', None)  # すべての列を表示
pd.set_option('display.max_rows', None)     # すべての行を表示（オプション）
#pd.set_option('display.width', None)        # 行の折り返しを防ぐ
pd.set_option('display.max_colwidth', None) # 列の内容を省略しない

In [ ]:
drop_cols=[
    "nl","el",'building_name_ruby', 'name_ruby', 'school_ele_code', 'school_jun_code', 'money_hoshou_company', 'free_rent_duration', 'free_rent_gen_timing'
    ]
land_price_cols=[
    "L02_001",#用途区分
    "L02_006",#調査価格
    "L02_024",#地積
    "L02_029",#水道
    "L02_037",#道路区分
    "L02_045",#最寄り駅までの道路距離
    "L02_055",
    "L02_056",
    "L02_057",
    "L02_058",
    "L02_059",
    "L02_060",
    "L02_061",
    "L02_062",
    "L02_063",
    "L02_064",
    "L02_065",
    "L02_066",
    "L02_067",
    "L02_068",
    "L02_069",
    "L02_070",
    "L02_071",
    "L02_072",
    "L02_073",
    "L02_074",
    "L02_075",
    "L02_076",
    "L02_077",
    "L02_078",
    "L02_079",
    "L02_080",
    "L02_081",
    "L02_082",
    "L02_083",
    "L02_084",
    "L02_085",
    "L02_086",
    "L02_087",
    "L02_088",
    "L02_089",
    "L02_090",
    "L02_091",
    "L02_092",
    "L02_093",
    "L02_094",
    "geometry"
    ]

In [6]:
gdf_json = gpd.read_file(os.path.join('..', 'data', 'train', 'L02-23.geojson'))#[land_price_cols]
gdf_json2 = gpd.read_file(os.path.join('..', 'data', 'train', 'L02-24.geojson'))#[land_price_cols]

# L02_098～L02_134のカラムを削除
drop_cols = [f'L02_{i:03d}' for i in range(98, 139)]
gdf_json = gdf_json.drop(columns=[col for col in drop_cols if col in gdf_json.columns], errors='ignore')
gdf_json2 = gdf_json2.drop(columns=[col for col in drop_cols if col in gdf_json2.columns], errors='ignore')

In [7]:
#train_gdf=utils.convert_to_geodataframe(train_df)
train_gdf=utils.nearest_merge(train_gdf,gdf_json,threshold=1500)
train_gdf=utils.nearest_merge(train_gdf,gdf_json2,threshold=1500)
#display(train_gdf.head())


In [15]:
train_df.columns

Index(['target_ym', 'money_room', 'building_id', 'building_status',
       'building_create_date', 'building_modify_date', 'building_type',
       'building_name', 'building_name_ruby', 'homes_building_name',
       ...
       'drugstore_distance', 'bank_distance', 'shopping_street_distance',
       'est_other_name', 'est_other_distance', 'statuses', 'parking_keiyaku',
       'money_hoshou_company', 'free_rent_duration', 'free_rent_gen_timing'],
      dtype='object', length=149)

In [ ]:
test_df = pd.read_csv(os.path.join('..', 'data', 'test', 'test.csv'))
test_gdf=utils.convert_to_geodataframe(test_df)
test_gdf=utils.nearest_merge(test_gdf,gdf_json,threshold=1500)
test_gdf["land_price_x"]=test_gdf.geometry.x
test_gdf["land_price_y"]=test_gdf.geometry.y
test_df = pd.DataFrame(test_gdf.drop(columns=[col for col in test_gdf.columns if 'geometry' in col]))

C:\Users\km20186\AppData\Local\Temp\ipykernel_32892\1463719098.py:1: DtypeWarning: Columns (46,55,56,63,146) have mixed types. Specify dtype option on import or set low_memory=False.
  test_df = pd.read_csv(os.path.join('..', 'data', 'test', 'test.csv'))


In [15]:
# distance列がNULLとなっている割合を計算
null_ratio = train_gdf['distance'].isnull().sum() / len(train_gdf)
print(f"distance列のNULL割合: {null_ratio:.2%} ({train_gdf['distance'].isnull().sum()}/{len(train_gdf)})")


distance列のNULL割合: 12.05% (43843/363924)


In [16]:
# GeoDataFrameをDataFrameに変換(geometry列を削除)
train_gdf["land_price_x"]=train_gdf.geometry.x
train_gdf["land_price_y"]=train_gdf.geometry.y
train_df = pd.DataFrame(train_gdf.drop(columns=[col for col in train_gdf.columns if 'geometry' in col]))

In [17]:
train, test = train_test_split(train_df, test_size=0.2, random_state=42)
train_data: pd.DataFrame = TabularDataset(train)
test_data: pd.DataFrame = TabularDataset(test)
model=TabularPredictor(
    label='money_room',
    problem_type='regression',
    eval_metric='mape'
)
predictor=model.fit(
    train_data=train_data,
    test_data=test_data,
    presets='medium_quality',
    time_limit=720,
)

No path specified. Models will be saved in: "AutogluonModels\ag-20260105_083437"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.4.0
Python Version:     3.10.19
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.26100
CPU Count:          20
Memory Avail:       13.14 GB / 31.66 GB (41.5%)
Disk Space Avail:   64.65 GB / 237.73 GB (27.2%)
Presets specified: ['medium_quality']
Using hyperparameters preset: hyperparameters='default'
Beginning AutoGluon training ... Time limit = 720s
AutoGluon will save models to "c:\Users\km20186\project\signature\my_code\AutogluonModels\ag-20260105_083437"
Train Data Rows:    291139
Train Data Columns: 353
Label Column:       money_room
Problem Type:       regression
Preprocessing data ...
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    15741.09 MB
	Train Data (Original)  Memory Usage: 292

[1000]	valid_set's l2: 2.80712e+13	valid_set's mean_absolute_percentage_error: -0.167856
[2000]	valid_set's l2: 2.53446e+13	valid_set's mean_absolute_percentage_error: -0.159816
[3000]	valid_set's l2: 2.39902e+13	valid_set's mean_absolute_percentage_error: -0.155437
[4000]	valid_set's l2: 2.3126e+13	valid_set's mean_absolute_percentage_error: -0.152642
[5000]	valid_set's l2: 2.25676e+13	valid_set's mean_absolute_percentage_error: -0.150633
[6000]	valid_set's l2: 2.21305e+13	valid_set's mean_absolute_percentage_error: -0.148701
[7000]	valid_set's l2: 2.17732e+13	valid_set's mean_absolute_percentage_error: -0.147228
[8000]	valid_set's l2: 2.14967e+13	valid_set's mean_absolute_percentage_error: -0.145926
[9000]	valid_set's l2: 2.13804e+13	valid_set's mean_absolute_percentage_error: -0.145125
[10000]	valid_set's l2: 2.12187e+13	valid_set's mean_absolute_percentage_error: -0.144184


	-0.1442	 = Validation score   (-mean_absolute_percentage_error)
	477.82s	 = Training   runtime
	0.59s	 = Validation runtime
Fitting model: LightGBM ... Training model for up to 120.87s of the 120.87s of remaining time.
	Fitting with cpus=14, gpus=0, mem=3.1/12.7 GB
	Ran out of time, early stopping on iteration 859. Best iteration is:
	[856]	valid_set's l2: 2.71013e+13	valid_set's mean_absolute_percentage_error: -0.163739
	-0.1637	 = Validation score   (-mean_absolute_percentage_error)
	121.56s	 = Training   runtime
	0.12s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ... Training model for up to 360.00s of the -1.24s of remaining time.
	Ensemble Weights: {'LightGBMXT': 0.938, 'LightGBM': 0.062}
	-0.144	 = Validation score   (-mean_absolute_percentage_error)
	0.02s	 = Training   runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 723.55s ... Best model: WeightedEnsemble_L2 | Estimated inference throughput: 4101.8 rows/s (2912 batch size)
TabularP

In [11]:
predictor.leaderboard(test)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBMXT,-0.146985,-0.143271,mean_absolute_percentage_error,15.98495,0.646749,592.879773,15.98495,0.646749,592.879773,1,True,1
1,WeightedEnsemble_L2,-0.146985,-0.143271,mean_absolute_percentage_error,16.06017,0.647752,592.891341,0.07522,0.001003,0.011568,2,True,2


In [54]:
predictor.leaderboard(test)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBMXT,-0.151821,-0.14987,mean_absolute_percentage_error,8.931070,0.428252,268.771355,8.931070,0.428252,268.771355,1,True,1
1,WeightedEnsemble_L2,-0.151821,-0.14987,mean_absolute_percentage_error,8.985064,0.429255,268.780988,0.053993,0.001003,0.009634,2,True,2


In [10]:
predictor.leaderboard(test)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBMXT,-0.150941,-0.148271,mean_absolute_percentage_error,6.924891,0.558865,619.218303,6.924891,0.558865,619.218303,1,True,1
1,WeightedEnsemble_L2,-0.150941,-0.148271,mean_absolute_percentage_error,6.976562,0.558865,619.222407,0.051672,0.000000,0.004104,2,True,2


In [47]:
test_data: pd.DataFrame = TabularDataset(test_df)
predictions = predictor.predict(test_data)
#

In [48]:
pred=pd.DataFrame(predictions)
pred.head()
pred.to_csv(os.path.join('..', 'data', 'submit', 'submission.csv'), index_label='id')

In [ ]:
train_data.head()

In [18]:
importance_df = predictor.feature_importance(test,num_shuffle_sets=1)

These features in provided data are not utilized by the predictor and will be ignored: ['building_name_ruby', 'name_ruby', 'school_ele_code', 'school_jun_code', 'money_hoshou_company', 'free_rent_duration', 'free_rent_gen_timing', 'A31a_102', 'A31a_104', 'A31a_107', 'A31a_108', 'L02_095_left', 'L02_017_right', 'L02_096_right']
Computing feature importance via permutation shuffling for 339 features using 5000 rows with 1 shuffle sets...
	873.22s	= Expected runtime (873.22s per shuffle set)
	783.43s	= Actual runtime (Completed 1 of 1 shuffle sets)


In [24]:
# カラム名(インデックス)に数字を含む行を除外
importance_df_d = importance_df[~importance_df.index.astype(str).str.contains(r'\d', regex=True)]

In [19]:
display(importance_df)

,importance,stddev,p_value,n,p99_high,p99_low
house_area,2.611348e-01,NaN,NaN,1,NaN,NaN
year_built,1.843354e-01,NaN,NaN,1,NaN,NaN
L02_006_right,8.527859e-02,NaN,NaN,1,NaN,NaN
L02_020_right,6.096032e-02,NaN,NaN,1,NaN,NaN
L02_006_left,4.915957e-02,NaN,NaN,1,NaN,NaN
L02_020_left,4.819189e-02,NaN,NaN,1,NaN,NaN
snapshot_land_area,3.929763e-02,NaN,NaN,1,NaN,NaN
rosen_name1,3.070851e-02,NaN,NaN,1,NaN,NaN
unit_area,2.109641e-02,NaN,NaN,1,NaN,NaN
madori_kind_all,1.966290e-02,NaN,NaN,1,NaN,NaN


In [ ]:
drop=["reform_place_other","reform_common_area","parking_keiyaku","est_other_distance",'building_name_ruby', 'name_ruby', 'school_ele_code', 'school_jun_code', 'money_hoshou_company', 'free_rent_duration', 'free_rent_gen_timing',]